In [1]:
import os
import sys
from pathlib import Path

# Add parent directory to Python path for imports
# This ensures we can import from _utils regardless of where the notebook is run from
current_dir = Path().resolve()
# If we're in the basics directory, go up one level; otherwise use current directory
if current_dir.name == 'basics':
    parent_dir = current_dir.parent
else:
    parent_dir = current_dir

if str(parent_dir) not in sys.path:
    sys.path.insert(0, str(parent_dir))

from openai import OpenAI
from _utils.website_scraper import fetch_website_contents
from _utils.standard_functions import load_env
from IPython.display import Markdown
from IPython.display import display, update_display


In [2]:
# reading the env
load_env()

api_key = os.getenv('OPENAI_API_KEY')
base_url = os.getenv("OPENAI_BASE_URL")

API key found and looks valid!


In [3]:
# setting up the usual objects
openai = OpenAI(base_url=base_url, api_key=api_key)
model = "gpt-5-nano"
website_to_review="https://bramptonbjj.com"

To summarize any website, we need to break down our problem into the following sub-steps:

1. Scan all the available links in the website map - using a python library                <Standard Programing>
2. Ask AI to ignore the irrelevant links and retain information about the relevant pages   <AI Intelligence>
3. AI will scan the link pages and grab the relevant content                               <AI Intelligence>
4. AI will then summarize that info and return the details                                 <AI Intelligence>

Step 1 - List all the weblinks for the supplied website

In [4]:
# custom function in the website_scrapper.py file
website_links = fetch_website_contents(website_to_review)
print(website_links)

No title found




We're going to ask the AI to fetch the details, by passing on instructions in the user prompt

In [5]:
user_prompt_prefix = """
Here are the contents of a website.
Provide a short summary of this website.
If it includes news or announcements, then summarize these too.
"""

In [17]:
system_prompt = """
You are a snarky assistant that analyzes the contents of a website,
and provides a short, snarky, humorous summary, ignoring text that might be navigation related.
Use a combination of paragraphs and bullet points.
Respond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.
"""

Like before, we'll create the messages object and call the OpenAI object

In [18]:
def messages(website):
    return [
        {"role": "system", "content": system_prompt},
        {"role": "user", "content": user_prompt_prefix + website}
    ]

In [8]:
messages(website_to_review)

[{'role': 'system',
  'content': '\nYou are a snarky assistant that analyzes the contents of a website,\nand provides a short, snarky, humorous summary, ignoring text that might be navigation related.\nRespond in markdown. Do not wrap the markdown in a code block - respond just with the markdown.\n'},
 {'role': 'user',
  'content': '\nHere are the contents of a website.\nProvide a short summary of this website.\nIf it includes news or announcements, then summarize these too.\nhttps://bramptonbjj.com'}]

We will use the similar implementation to call OpenAPI as before, but we'll just wrap it in a function to make it easier for us

In [9]:
def summarize_website(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(model = model, messages = messages(website))
    return response.choices[0].message.content

In [11]:
summarize_website(website_to_review)

'- A Brampton Brazilian Jiu Jitsu Academy that basically sells adrenaline and a free trial: sign up for a semi-private intro plus a loaner uniform.\n- It markets a wide class range for kids, adults, beginners, and pro competitors, with a schedule spanning early morning to evening.\n- Locations listed: Brampton main location, plus Georgetown and Orangeville sites.\n- Contact info included (address, email, phone) and there’s a form to book the free trial.\n- A few “news/announcements” vibes exist, but the only dated item is 14-12-19 about booking a free trial—yep, ancient history.\n- Extra stuff: standard marketing fluff like “Fostering positive lifestyles,” a reCAPTCHA/privacy policy note, and navigation clutter that screams “we have more pages than content.”'

In [12]:
# Make it presentable
def summarize_website_cleanly(url):
    website = fetch_website_contents(url)
    response = openai.chat.completions.create(model = model, messages = messages(website))
    return display(Markdown(response.choices[0].message.content))

In [19]:
summarize_website_cleanly(website_to_review)

Brampton Brazilian Jiu Jitsu Academy is a gym that promises “fostering positive lifestyles through Brazilian Jiu Jitsu.” They lure newcomers with a free trial that includes a semi-private intro and a loaner uniform, which is basically the gym’s way of saying “trust us, you’ll look cool in our gear.” The site keeps things beginner-friendly and claims to offer classes for kids, adults, and pros, with a schedule meant to fit busy lives.

What it’s selling
- Free trial: semi-private lesson + loaner gi, perfect for BJJ newbies.
- Class lineup: wide range for kids, adults, men, women, zero-experience folks, and competitive athletes.
- Flexible schedule: early morning, noon, and evening classes to fit around real lives.
- Locations: Brampton with mentions of Georgetown and Orangeville locations.
- Getting in touch: contact info, a booking form, and a standard privacy/terms disclaimer.

Notable quirks (and what’s missing)
- The site repeats navigation items a lot, which is… amusingly redundant.
- The only apparent “news” is a dated line from 14-12-19 on a booking prompt; no current announcements are visible.
- Details like instructor bios or exact price lists aren’t included in the text you shared.

Contact and basics
- Address: 1295 Williams Pkwy E, Brampton, ON L6S 3J8, Canada
- Email: info@bramptonbjj.ca
- Phone: 416-230-9653
- Extras: a reCAPTCHA/privacy terms note, which is standard but never exciting.

In short: if you’re in Brampton (or nearby Georgetown/Orangeville) and want a beginner-friendly BJJ gym with a free trial, this place has you covered—just beware the site’s nostalgia for 2019.

In [14]:
# Make a fancy summary
def summarize_website_fancy(url):
    website = fetch_website_contents(url)
    stream = openai.chat.completions.create(model = model, messages = messages(website), stream=True)

    response = ""

    display_handle = display(Markdown(""), display_id=True)
    for chunk in stream:
        response += chunk.choices[0].delta.content or ''
        update_display(Markdown(response), display_id=display_handle.display_id)

In [20]:
summarize_website_fancy(website_to_review)

Here’s the gist of the Brampton Brazilian Jiu Jitsu Academy site, with a splash of sass.

- It’s a Brazilian Jiu-Jitsu academy that’s pitching itself as a lifestyle choice, not just a gym. The motto: “Fostering positive lifestyles through Brazilian Jiu Jitsu.”

- They push a free trial heavy on the perks: a semi-private intro lesson plus a loaner uniform, which is nice if you don’t own a gi yet.

- Classes are all-ages and all-skill levels, from kids and beginners to pros, with a schedule that tries to fit around real people’s lives (early morning, noon, and evening slots).

- Locations mentioned include Brampton (the main one), plus Georgetown and Orangeville, suggesting a little regional empire.

- Contact details are provided: address (1295 Williams Pkwy E, Brampton), email, and phone number.

- The site includes the usual menu stuff you expect: price list, instructors, schedule, and contact information. There’s a sign-up form with a recaptcha/privacy policy hook, just in case you’re suspicious of free trials.

News/Announcements:
- There isn’t any fresh news on display. The only dated item is a “14-12-19 book your free trial,” which reads more as an old promo than a current announcement.

Bottom line: If you want a BJJ home with a generous free-trial offer and flexible class times, this site is your brochure. If you’re looking for up-to-the-minute news, you’re probably going to have to call or email.